# StopsWords

In [1]:
import nltk

from pyspark.ml.feature import StopWordsRemover, Tokenizer
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder
                     .master('local[*]') \
                     .appName('spark_application') \
                     .config("spark.sql.repl.eagerEval.enabled", True)  # eval DataFrame in notebooks
                     .getOrCreate())

sc = spark.sparkContext
print(f'Spark version: {spark.version}')

Spark version: 3.5.1


In [3]:
df = spark.createDataFrame([
        "Eso es pan comido",
        "A cada cerdo le llega su San Martín",
        "Dormir la mona",
        "Tener algo en la punta de la lengua",
        "Quedarse con la boca abierta"
     ], "string").toDF("text")
df.show(truncate=False)

+-----------------------------------+
|text                               |
+-----------------------------------+
|Eso es pan comido                  |
|A cada cerdo le llega su San Martín|
|Dormir la mona                     |
|Tener algo en la punta de la lengua|
|Quedarse con la boca abierta       |
+-----------------------------------+



In [4]:
nltk.download("stopwords")
stops_spanish = nltk.corpus.stopwords.words('spanish')
print(stops_spanish)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacqueline\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Text to tokens
df = df.drop('text_tokens', 'token_without_stop_words')
df = Tokenizer(inputCol="text", outputCol="text_tokens").transform(df)

# Removing stopwords
stopwords = StopWordsRemover(inputCol='text_tokens', outputCol='token_without_stop_words', stopWords=stops_spanish)
df = stopwords.transform(df)
df.show(truncate=False)

+-----------------------------------+--------------------------------------------+---------------------------------+
|text                               |text_tokens                                 |token_without_stop_words         |
+-----------------------------------+--------------------------------------------+---------------------------------+
|Eso es pan comido                  |[eso, es, pan, comido]                      |[pan, comido]                    |
|A cada cerdo le llega su San Martín|[a, cada, cerdo, le, llega, su, san, martín]|[cada, cerdo, llega, san, martín]|
|Dormir la mona                     |[dormir, la, mona]                          |[dormir, mona]                   |
|Tener algo en la punta de la lengua|[tener, algo, en, la, punta, de, la, lengua]|[tener, punta, lengua]           |
|Quedarse con la boca abierta       |[quedarse, con, la, boca, abierta]          |[quedarse, boca, abierta]        |
+-----------------------------------+---------------------------

In [6]:
spark.stop()